In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

from ember_utils import *
from ember_model import *
from ember_pjr_utils import *



In [3]:
data_path = '../../BODMAS_DATA/'

filename = data_path + 'XY_processed.npz'
data = np.load(filename)
X = data['X']  # all the feature vectors
y = data['Y']  # labels, 0 as benign, 1 as malicious

In [8]:
exp_seeds = [random.randint(1, 99999) for i in range(10)]


accs_all = []
rocauc_all = []


num_epoch = 500
batch_size = 256
patience = 5

replay_type, current_task = 'bodmas', 'bodmas_joint'

for exp in exp_seeds:
    

    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=exp)
    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    


    standardization = StandardScaler()
    standard_scaler = standardization.fit(X_train)

    X_train = standard_scaler.transform(X_train)
    X_test = standard_scaler.transform(X_test)


    X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
    X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)  

    
    
    model_save_dir = '../bodmas_model/'
    create_parent_folder(model_save_dir)

    opt_save_path = '../bodmas_model/'
    create_parent_folder(opt_save_path)

    print(f'X_train {X_train.shape} Y_train {Y_train.shape}')
    print(f'X_test {X_test.shape} Y_test {Y_test.shape}')
    
    
    task_training_time, epoch_ran, training_loss, validation_loss  = training_early_stopping(\
                                 model, model_save_dir, opt_save_path, X_train, Y_train,\
                                 X_test, Y_test, patience, batch_size, device, optimizer, num_epoch,\
                                 criterion, replay_type, current_task, exp, earlystopping=True)

    
    end_time = time.time()

    print(f'Elapsed time {(end_time - start_time)/60} mins.') 
    
    acc, rocauc = testing_aucscore(model, X_test, Y_test, batch_size, device)
    print()
    accs_all.append(acc)
    rocauc_all.append(rocauc)



Torch 1.6.0 CUDA 10.2
Model has 3.132161m parameters


  0%|          | 0/434 [00:00<?, ?it/s]

X_train (111333, 2381) Y_train (111333,)
X_test (12371, 2381) Y_test (12371,)
Epoch 1 of 500


  0%|          | 0/434 [00:00<?, ?it/s]

Train Loss: 0.1133, Train Acc: 0.9539
Val Loss: 0.0425, Val Acc: 0.9903
Validation loss decreased (inf --> 0.042486).  Saving model ...
../bodmas_model/best_model_epoch_1.pt
../bodmas_model/best_optimizer_epoch_1.pt
Epoch 2 of 500


...

Train Loss: 0.0062, Train Acc: 0.9982
Val Loss: 0.0206, Val Acc: 0.9949
EarlyStopping counter: 4 out of 5
Epoch 16 of 500


  0%|          | 0/49 [00:00<?, ?it/s]

Train Loss: 0.0055, Train Acc: 0.9984
Val Loss: 0.0201, Val Acc: 0.9953
EarlyStopping counter: 5 out of 5
Early stopping
Training time: 1.057 minutes
Elapsed time 1.1583699544270833 mins.


100%|██████████| 49/49 [00:00<00:00, 87.24it/s] 

test accuracy 0.9952898317453812 and ROC-AUC 0.9953749561533511



In [12]:
np.mean(rocauc_all), np.std(rocauc_all), np.mean(accs_all), np.std(accs_all)

(0.9952213555622429,
 0.000742324851330087,
 0.9952055019991739,
 0.0007683991448096392)